In [28]:
import numpy as np
def randomize_binary_network(observed_matrix, preserve_degree=True, iterations=1000, seed=None):
    """
    Randomizes a binary adjacency matrix while preserving its structure.

    Parameters:
    observed_matrix (np.ndarray): Binary adjacency matrix (NxN).
    preserve_degree (bool): Whether to preserve the degree distribution.
    iterations (int): Number of edge swaps to perform.
    seed (int, optional): Random seed for reproducibility.

    Returns:
    np.ndarray: A randomized binary adjacency matrix.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Ensure the input matrix is binary
    if not np.array_equal(observed_matrix, observed_matrix.astype(bool)):
        raise ValueError("The observed matrix must be binary (0s and 1s).")
    
    # Ensure symmetry for undirected networks
    observed_matrix = np.triu(observed_matrix) + np.triu(observed_matrix, 1).T
    randomized_matrix = observed_matrix.copy()

    if preserve_degree:
        # Get all edges (i, j) where i < j (upper triangular matrix)
        edges = np.array(np.transpose(np.triu_indices_from(observed_matrix, k=1)))

        # Perform edge swaps to randomize the network
        for _ in range(iterations):
            # Select two random edges to swap
            idx1, idx2 = np.random.choice(len(edges), 2, replace=False)
            a, b = edges[idx1]
            c, d = edges[idx2]

            # Ensure no duplicate edges or self-loops after rewiring
            if len({a, b, c, d}) == 4 and randomized_matrix[a, d] == 0 and randomized_matrix[c, b] == 0:
                # Swap edges: (a, b) with (c, d) -> (a, d) and (c, b)
                randomized_matrix[a, b] = 0
                randomized_matrix[b, a] = 0
                randomized_matrix[c, d] = 0
                randomized_matrix[d, c] = 0

                randomized_matrix[a, d] = 1
                randomized_matrix[d, a] = 1
                randomized_matrix[c, b] = 1
                randomized_matrix[b, c] = 1

                # Update the edge list
                edges[idx1] = [a, d]
                edges[idx2] = [c, b]
    else:
        # Completely randomize connections while preserving density
        num_edges = np.sum(np.triu(observed_matrix, k=1))  # Count total connections
        randomized_matrix.fill(0)
        
        possible_edges = np.transpose(np.triu_indices_from(observed_matrix, k=1))
        selected_edges = possible_edges[np.random.choice(len(possible_edges), num_edges, replace=False)]
        
        for edge in selected_edges:
            i, j = edge
            randomized_matrix[i, j] = 1
            randomized_matrix[j, i] = 1  # Keep symmetry

    return randomized_matrix


In [30]:
# Example observed correlation matrix (randomly generated for demonstration)

file_path = R'C:\GIT\Connectomics\Research_Project\Binary_Notebooks\Binary_Output\10%\Matrices\Caltech_0051456_rois_cc400_mat.txt'

observed_matrix = np.loadtxt(file_path)

randomized_matrix = randomize_binary_network(observed_matrix, preserve_degree=True, iterations=1000, seed=42)
import networkx as nx

G_O = nx.from_numpy_array(observed_matrix)
G_R = nx.from_numpy_array(randomized_matrix)


print(nx.global_efficiency(G_O))
print(nx.global_efficiency(G_R))

print("Avg Clustering")
print(nx.average_clustering(G_O))
print(nx.average_clustering(G_R))

0.3879344905666128
0.5189108251996218
Avg Clustering
0.5263222351187967
0.3395925883340588
